In [2]:
from bs4 import BeautifulSoup 
import requests 
import pandas as pd

In [3]:
Headers = ({
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5'
})


In [4]:
mainUrl="https://www.amazon.com/s?k=books&i=stripbooks-intl-ship&crid=3K7E49HWHO3ZK&sprefix=book%2Cstripbooks-intl-ship%2C356&ref=nb_sb_noss_1"

In [5]:
mainPage=requests.get(mainUrl, headers=Headers)

In [6]:
mainPage

<Response [200]>

In [7]:
mainSoup=BeautifulSoup(mainPage.content, 'html.parser')

In [8]:
links=mainSoup.find_all("a",attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

In [9]:
b=links[2].get('href')

In [10]:
Link='http://amazon.com' +b

In [11]:
Link

'http://amazon.com/Good-Energy-Surprising-Connection-Metabolism/dp/0593712641/ref=sr_1_3?crid=3K7E49HWHO3ZK&dib=eyJ2IjoiMSJ9.kTpsfyQRHnXmAZfjAWl-oxt3Gdth497qMdfI_6vDl78K8DS26FcURgzpe-WRDnil5pOX9hVlN-GF1eEkuxm09VSf50wxLW4uiEKj55cMlRvCfeR_ncslmUdwKl87LqZ-32gLLXbqghY4VgqNz0rHJBnNmNSTFfHHWHf1uuF44Ef_-9lTjLSWyAX_BBs7rJd8vSiWoF6caAG5bmuKU3IdSMjYorruBp1WONEpdSzoDbc.3444bTcE2m_2Jfipx769rLSD0yJuVAWBYahkD5faFNc&dib_tag=se&keywords=books&qid=1730734184&s=books&sprefix=book%2Cstripbooks-intl-ship%2C356&sr=1-3'

In [12]:
b1=requests.get(Link, headers=Headers)

In [13]:
import csv


In [14]:

all_books_data = {}

for i in links[0:20]:  
    b=i.get('href')
    Link='http://amazon.com' +b
    b1=requests.get(Link, headers=Headers)
    # convert to html
    soup = BeautifulSoup(b1.content, 'html.parser')
    
    # Find all review divs
    reviews = soup.find_all('div', class_='a-section review aok-relative')
    # Temporary storage for current book's reviews
    book_reviews = {}
    
    # saving data
    customer_data = []
    
    # eaxtract each customer and his rating 
    for review in reviews:
        # Extract customer name
        name = review.find('span', class_='a-profile-name').get_text(strip=True)
        
        # Extract rating
        rating = review.find('i', {'data-hook': 'review-star-rating'}).find('span').get_text(strip=True)
        
        # store
        customer_data.append({'name': name, 'rating': rating})
    
    
    print(soup.find('span',attrs={'id':"productTitle"}).text.strip())
    for data in customer_data:
        print(f"Customer Name: {data['name']}, Rating: {data['rating'][0:3]}")


Melania
Customer Name: Gloria V., Rating: 5.0
Customer Name: Marie Valerie, Rating: 5.0
Customer Name: Carol O, Rating: 5.0
Customer Name: Kathleen Edelmuth, Rating: 4.0
Customer Name: Laura M, Rating: 5.0
Customer Name: Kindle Customer, Rating: 5.0
Hot Mess (Diary of a Wimpy Kid Book 19) (Diary of a Wimpy Kid, 19) (Volume 19)
Customer Name: rahul-java, Rating: 5.0
Customer Name: susan, Rating: 5.0
Customer Name: Julian, Rating: 5.0
Customer Name: kaseyatbat, Rating: 5.0
Customer Name: S.C., Rating: 4.0
Customer Name: Mrs. J, Rating: 5.0
Customer Name: wsw, Rating: 5.0
Customer Name: sun007set, Rating: 5.0
Good Energy: The Surprising Connection Between Metabolism and Limitless Health
Customer Name: Tui, Rating: 5.0
Customer Name: JMN, Rating: 5.0
Customer Name: Debs Stuff, Rating: 5.0
Customer Name: Patti, Rating: 5.0
Customer Name: Fudge13, Rating: 4.0
The Coming Golden Age: 31 Ways to be Kingdom Ready
Customer Name: HercNDolly, Rating: 5.0
Customer Name: DAI, Rating: 5.0
Customer Nam

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

#customer data across multiple books
customer_ratings = {}

# Loop over each link
for i in links[0:20]:
    b = i.get('href')
    Link = 'http://amazon.com' + b
    b1 = requests.get(Link, headers=Headers)
    
    # Convert to HTML
    soup = BeautifulSoup(b1.content, 'html.parser')
    
    # Get the book title
    book_title = soup.find('span', attrs={'id': "productTitle"}).text.strip()
    
    # Find all review divs
    reviews = soup.find_all('div', class_='a-section review aok-relative')
    
    # Extract each customer and their rating for this book
    for review in reviews:
        # Extract customer name
        name = review.find('span', class_='a-profile-name').get_text(strip=True)
        
        # Extract rating, keeping only the numeric part (e.g., '5.0')
        rating = review.find('i', {'data-hook': 'review-star-rating'}).find('span').get_text(strip=True)[:3]
        
        # Initialize customer's entry if not already present
        if name not in customer_ratings:
            customer_ratings[name] = {}
        
        # Add the book rating to this customer's entry
        customer_ratings[name][book_title] = rating

# Get all unique book titles for the header row
all_books = sorted({title for reviews in customer_ratings.values() for title in reviews})

# Save the data to the specified file path
csv_path = '/Users/apple/Desktop/Last_year/IRS/222102535Karim_assignment1/222102535karim_c_reviews'
with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write header row with book titles
    header = ['Customer Name'] + all_books
    writer.writerow(header)
    
    # Write each customer's name and ratings for each book
    for customer, ratings in customer_ratings.items():
        row = [customer]  # Start with the customer name
        for book in all_books:
            row.append(ratings.get(book, 'N/A'))  # Add rating if available, else 'N/A'
        writer.writerow(row)

print("done")


In [20]:
import pandas as pd
import numpy as np

# Load the data from the provided CSV file
df = pd.read_csv('/Users/apple/Desktop/Last_year/IRS/222102535Karim_assignment1/222102535Karim_customers.csv')


In [21]:
# Display the columns in the DataFrame to identify the correct column for NaN replacement
df.columns


Index(['Customer Name',
       'A Little SPOT of Emotion 8 Book Box Set (Books 1-8: Anger, Anxiety, Peaceful, Happiness, Sadness, Confidence, Love, & Scribble Emotion)',
       'A Navy SEAL's Bug-In Guide: How to Turn Your House into the Safest Place on Earth',
       'Atomic Habits: An Easy & Proven Way to Build Good Habits & Break Bad Ones',
       'Dad, I Want to Hear Your Story: A Father’s Guided Journal To Share His Life & His Love (Hear Your Story Books)',
       'Framed: Astonishing True Stories of Wrongful Convictions',
       'From Here to the Great Unknown: A Memoir',
       'Good Energy: The Surprising Connection Between Metabolism and Limitless Health',
       'Hot Mess (Diary of a Wimpy Kid Book 19) (Diary of a Wimpy Kid, 19) (Volume 19)',
       'Jobs to Be Done: A Roadmap for Customer-Centered Innovation',
       'Melania', 'Patriot: A Memoir',
       'Relentless Sales: The Skills, Mentality, & Faith Needed to Be Great in Sales',
       'Spooky Cutie: Coloring Book for A

In [23]:
# Identify book columns (all columns except the first, which is 'Customer Name')
book_columns = df.columns[1:]

# Replace NaN values in each book column with a random choice from 1, 2, 3, 4, 5, and NaN
df[book_columns] = df[book_columns].applymap(lambda x: np.random.choice([1, 2, 3, 4, 5, np.nan]) if pd.isna(x) else x)

# Save the updated DataFrame to a new CSV file
new_csv_path = '/Users/apple/Desktop/Last_year/IRS/222102535Karim_assignment1/222102535karim_Preprosising_reviews.csv'
df.to_csv(new_csv_path, index=False)

# Display the updated DataFrame
df.head()


/var/folders/2t/bv9pyvvs1y39zktfxmgd8bnc0000gn/T/ipykernel_1168/2470075482.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[book_columns] = df[book_columns].applymap(lambda x: np.random.choice([1, 2, 3, 4, 5, np.nan]) if pd.isna(x) else x)


,Customer Name,"A Little SPOT of Emotion 8 Book Box Set (Books 1-8: Anger, Anxiety, Peaceful, Happiness, Sadness, Confidence, Love, & Scribble Emotion)",A Navy SEAL's Bug-In Guide: How to Turn Your House into the Safest Place on Earth,Atomic Habits: An Easy & Proven Way to Build Good Habits & Break Bad Ones,"Dad, I Want to Hear Your Story: A Father’s Guided Journal To Share His Life & His Love (Hear Your Story Books)",Framed: Astonishing True Stories of Wrongful Convictions,From Here to the Great Unknown: A Memoir,Good Energy: The Surprising Connection Between Metabolism and Limitless Health,"Hot Mess (Diary of a Wimpy Kid Book 19) (Diary of a Wimpy Kid, 19) (Volume 19)",Jobs to Be Done: A Roadmap for Customer-Centered Innovation,Melania,Patriot: A Memoir,"Relentless Sales: The Skills, Mentality, & Faith Needed to Be Great in Sales",Spooky Cutie: Coloring Book for Adults and Teens Featuring Adorable Creepy Creatures in Cozy Hygge Moments for Relaxation (Cozy Spaces Coloring),The Anxious Generation: How the Great Rewiring of Childhood Is Causing an Epidemic of Mental Illness,The Coming Golden Age: 31 Ways to be Kingdom Ready,The Grey Wolf: A Novel (Chief Inspector Gamache Novel Book 19),The Message,"The Wingmen: The Unlikely, Unusual, Unbreakable Friendship Between John Glenn and Ted Williams",War
0,Gloria V.,1.0,2.0,2.0,3.0,3.0,1.0,2.0,5.0,2.0,5.0,5.0,3.0,3.0,1.0,3.0,4.0,2.0,4.0,4.0
1,T. J. Wells,1.0,2.0,2.0,4.0,3.0,2.0,1.0,2.0,4.0,5.0,3.0,4.0,3.0,1.0,1.0,4.0,4.0,NaN,3.0
2,Laura M,5.0,3.0,NaN,5.0,NaN,5.0,4.0,1.0,NaN,5.0,3.0,2.0,1.0,4.0,2.0,3.0,5.0,3.0,2.0
3,Kathleen Edelmuth,3.0,4.0,3.0,1.0,3.0,3.0,NaN,1.0,3.0,4.0,3.0,3.0,5.0,5.0,2.0,4.0,1.0,4.0,2.0
4,Marie Valerie,4.0,3.0,1.0,4.0,2.0,3.0,2.0,3.0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,3.0,4.0,5.0


In [29]:
import pandas as pd
from scipy.spatial.distance import cosine

# Load the data from the specified CSV file
csv_path = '/Users/apple/Desktop/Last_year/IRS/222102535Karim_assignment1/222102535karim_Preprosising_reviews.csv'
df = pd.read_csv(csv_path)

# Set "Customer Name" as the index to easily access rows by customer name
df.set_index('Customer Name', inplace=True)

# Get the books rated by "Laura M" (non-NaN values)
laura_ratings = df.loc['Laura M'].dropna()
laura_books = laura_ratings.index

# Define the threshold for similarity (90% of books rated by Laura M)
min_common_books = int(0.9 * len(laura_books))

# Dictionary to store similarity scores
similarity_scores = {}

for customer in df.index:
    # Skip "Laura M" herself
    if customer == 'Laura M':
        continue
    
    # Get ratings for the current customer on Laura's rated books
    customer_ratings = df.loc[customer, laura_books].dropna()
    
    # Check if the customer has rated at least 90% of the books Laura rated
    if len(customer_ratings) >= min_common_books:
        # Align the ratings of "Laura M" and the current customer for these common books
        aligned_ratings = laura_ratings[customer_ratings.index]
        
        # Calculate cosine similarity (1 - cosine distance) between Laura's and the customer's ratings
        similarity = 1 - cosine(aligned_ratings, customer_ratings)
        
        # Add to the similarity scores dictionary
        similarity_scores[customer] = similarity

# Sort by similarity and take the top 10 customers
top_10_similar_customers = sorted(similarity_scores, key=similarity_scores.get, reverse=True)[:10]

# Include "Laura M" and her top 10 similar customers
selected_customers = ['Laura M'] + top_10_similar_customers
filtered_df = df.loc[selected_customers]

# Save the filtered data to a new CSV file
filtered_csv_path = '/Users/apple/Desktop/Last_year/IRS/222102535Karim_assignment1/222102535karim_Top10_Similar_Customers.csv'
filtered_df.to_csv(filtered_csv_path)

# Display path to the saved file
filtered_csv_path


'/Users/apple/Desktop/Last_year/IRS/222102535Karim_assignment1/222102535karim_Top10_Similar_Customers.csv'

In [9]:
import pandas as pd
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr
import numpy as np

# Load the data
csv_path = '/Users/apple/Desktop/Last_year/IRS/222102535Karim_assignment1/222102535karim_Top10_Similar_Customers.csv'
df = pd.read_csv(csv_path)
df.set_index('Customer Name', inplace=True)

# Function to calculate cosine similarity between two users
def cosine_similarity(user1, user2):
    # Drop NaN values to align ratings on common books only
    common_ratings = user1.dropna().index.intersection(user2.dropna().index)
    if len(common_ratings) == 0:
        return 0  # No similarity if no common books rated
    
    u1 = user1[common_ratings]
    u2 = user2[common_ratings]
    
    # Calculate cosine similarity
    return 1 - cosine(u1, u2)

# Function to calculate Pearson correlation between two users
def pearson_similarity(user1, user2):
    common_ratings = user1.dropna().index.intersection(user2.dropna().index)
    if len(common_ratings) == 0:
        return 0  # No similarity if no common books rated
    
    u1 = user1[common_ratings]
    u2 = user2[common_ratings]
    
    # Calculate Pearson correlation
    corr, _ = pearsonr(u1, u2)
    return corr if not np.isnan(corr) else 0

# Predict missing values for Laura M using the specified similarity measure
def predict_missing_ratings(df, target_customer, similarity_func):
    target_ratings = df.loc[target_customer]
    missing_books = target_ratings[target_ratings.isna()].index
    
    predictions = {}
    
    for book in missing_books:
        # Weighted sum numerator and denominator
        weighted_sum = 0
        sim_sum = 0
        
        # Loop through other users to find similarity and their rating for the book
        for other_customer in df.index:
            if other_customer == target_customer or pd.isna(df.loc[other_customer, book]):
                continue
            
            # Calculate similarity
            similarity = similarity_func(target_ratings, df.loc[other_customer])
            
            # Accumulate weighted sum and similarity sum
            weighted_sum += similarity * df.loc[other_customer, book]
            sim_sum += abs(similarity)
        
        # Calculate the predicted rating
        predictions[book] = weighted_sum / sim_sum if sim_sum != 0 else np.nan
    
    return predictions

# Predict missing ratings for Laura M using Cosine and Pearson similarities
cosine_predictions = predict_missing_ratings(df, 'Laura M', cosine_similarity)
pearson_predictions = predict_missing_ratings(df, 'Laura M', pearson_similarity)

# Output the predictions
cosine_predictions, pearson_predictions


({'Atomic Habits: An Easy & Proven Way to Build Good Habits & Break Bad Ones': 2.493583552495079,
  'Framed: Astonishing True Stories of Wrongful Convictions': 2.691143451711679,
  'Jobs to Be Done: A Roadmap for Customer-Centered Innovation': 4.0998008678045235},
 {'Atomic Habits: An Easy & Proven Way to Build Good Habits & Break Bad Ones': 2.370868134174206,
  'Framed: Astonishing True Stories of Wrongful Convictions': 2.4128669921086896,
  'Jobs to Be Done: A Roadmap for Customer-Centered Innovation': 3.9716772793208026})

In [3]:
import pandas as pd

# Define the path to your data
csv_path = '/Users/apple/Desktop/Last_year/IRS/222102535Karim_assignment1/222102535karim_Top10_Similar_Customers.csv'

# Load the dataset
df = pd.read_csv(csv_path)

# Function to calculate average rating for each customer, skipping NaN values
def calculate_average_ratings(df):
    avg_ratings = {}
    for index, row in df.iterrows():
        customer_name = row['Customer Name']
        ratings = row[1:]  # Get all ratings (excluding the customer name)
        valid_ratings = ratings.dropna()  # Drop NaN values
        avg_rating = valid_ratings.mean() if not valid_ratings.empty else None
        avg_ratings[customer_name] = avg_rating
    return avg_ratings

# Calculate and display average ratings
average_ratings = calculate_average_ratings(df)
for customer, avg_rating in average_ratings.items():
    print(f"{customer}: {avg_rating:.2f}" if avg_rating is not None else f"{customer}: No ratings available")


Laura M: 3.31
Kindle Customer MJ: 2.89
HuntleyMC: 2.68
The Littles: 3.35
Markus Robbins: 3.28
Bruce_in_LA: 3.44
DAI: 4.05
Heather Key: 3.42
Kristen Stoeger: 2.74
Nick Hutchison: 3.42
Debs Stuff: 2.89


In [8]:
import pandas as pd
import numpy as np

# Define the path to your data
csv_path = '/Users/apple/Desktop/Last_year/IRS/222102535Karim_assignment1/222102535karim_Top10_Similar_Customers.csv'

# Load the dataset
df = pd.read_csv(csv_path)

# Specify the items of interest
items = [
    "Atomic Habits: An Easy & Proven Way to Build Good Habits & Break Bad Ones",
    "Jobs to Be Done: A Roadmap for Customer-Centered Innovation",
    "Framed: Astonishing True Stories of Wrongful Convictions"
]

# Step 1: Calculate the average rating for each user
user_averages = df.iloc[:, 1:].mean(axis=1)

# Step 2: Adjust each rating by subtracting the user's average rating
adjusted_ratings = df.iloc[:, 1:].sub(user_averages, axis=0)

# Function to calculate adjusted cosine similarity between two items
def adjusted_cosine_similarity(item1_ratings, item2_ratings):
    # Select ratings where both items have ratings (non-NaN)
    mask = ~item1_ratings.isna() & ~item2_ratings.isna()
    item1_ratings = item1_ratings[mask]
    item2_ratings = item2_ratings[mask]
    
    # Calculate the adjusted cosine similarity
    numerator = (item1_ratings * item2_ratings).sum()
    denominator = np.sqrt((item1_ratings ** 2).sum()) * np.sqrt((item2_ratings ** 2).sum())
    return numerator / denominator if denominator != 0 else 0

# Step 3: Function to compute similarities of a target item with all other items
def compute_item_similarities(target_item):
    item_similarities = {}
    for column in adjusted_ratings.columns:
        if column != target_item:
            similarity = adjusted_cosine_similarity(adjusted_ratings[target_item], adjusted_ratings[column])
            item_similarities[column] = similarity
    return item_similarities

# Step 4: Function to estimate missing ratings for a target item based on similar items
def estimate_missing_ratings(df, target_item, item_similarities):
    estimated_ratings = []
    for i, row in df.iterrows():
        if pd.isna(row[target_item]):  # Only estimate for missing values
            # Calculate weighted average of available ratings for similar items
            weighted_sum = sum(row[col] * item_similarities[col] for col in item_similarities if not pd.isna(row[col]))
            similarity_sum = sum(item_similarities[col] for col in item_similarities if not pd.isna(row[col]))
            estimated_rating = weighted_sum / similarity_sum if similarity_sum != 0 else np.nan
            estimated_ratings.append(estimated_rating)
        else:
            estimated_ratings.append(row[target_item])  # Keep existing rating if present
    return estimated_ratings

# Step 5: Apply the function to estimate missing values for each item of interest
for item in items:
    item_similarities = compute_item_similarities(item)
    df[item] = estimate_missing_ratings(df, item, item_similarities)

# Display the resulting DataFrame with estimated missing values
print(df[items])


    Atomic Habits: An Easy & Proven Way to Build Good Habits & Break Bad Ones  \
0                                            6.573325                           
1                                            1.000000                           
2                                            3.000000                           
3                                            2.000000                           
4                                            3.000000                           
5                                            2.000000                           
6                                            2.000000                           
7                                            5.000000                           
8                                            1.000000                           
9                                            3.000000                           
10                                           3.000000                           

    Jobs to Be Done: A Road

In [47]:
#OLD CODES

import pandas as pd

# Path to the CSV file
csv_path = '/Users/apple/Desktop/Html/customers.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_path)

# Identify customers with ratings for more than one book
multi_book_customers = df[df.notna().sum(axis=1) > 2]  # >2 because first column is 'Customer Name'

# Check if we have at least 5 customers, otherwise add more customers
if len(multi_book_customers) < 5:
    additional_customers = df[df.notna().sum(axis=1) > 1].drop(multi_book_customers.index)
    required_count = 5 - len(multi_book_customers)
    multi_book_customers = pd.concat([multi_book_customers, additional_customers.head(required_count)])

# Reset index for the resulting DataFrame
multi_book_customers.reset_index(drop=True, inplace=True)

# Save the filtered DataFrame to the specified path
output_path = '/Users/apple/Desktop/Html/customers_2.csv'
multi_book_customers.to_csv(output_path, index=False)

print(f"Filtered data saved successfully at {output_path}.")


Filtered data saved successfully at /Users/apple/Desktop/Html/customers_2.csv.


In [ ]:
import pandas as pd

# Path to the CSV file
csv_path = '/Users/apple/Desktop/Html/customers_2.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_path)

# Loop until we have 5 columns or fewer
while df.shape[1] > 5:
    # Drop columns that have all values as NaN
    df.dropna(axis=1, how='all', inplace=True)

# Save the cleaned DataFrame to the specified path
cleaned_csv_path = '/Users/apple/Desktop/Html/customers_3.csv'
df.to_csv(cleaned_csv_path, index=False)

print(f"Cleaned data saved successfully at {cleaned_csv_path}. Remaining columns: {df.shape[1]}.")


In [42]:
import pandas as pd
import numpy as np

# Path to the CSV file
csv_path = '/Users/apple/Desktop/Html/customers_3.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_path)

# Fill in the missing values
num_customers = df.shape[0]

# Ensure the first customer remains unchanged
# Fill all values for the second customer with random integers from 1 to 5
if num_customers > 1:
    df.iloc[1, 1:] = np.random.randint(1, 6, size=df.shape[1] - 1)

# Fill remaining missing values for other customers
for i in range(2, num_customers):  # Start from the third customer
    # Get the indices of columns with missing values (excluding name)
    missing_indices = df.columns[1:][df.iloc[i, 1:].isna()].tolist()
    
    # Ensure we only leave one missing value
    if missing_indices:
        # Randomly select one column to leave missing
        idx_to_leave_missing = np.random.choice(missing_indices)
        for idx in missing_indices:
            if idx != idx_to_leave_missing:
                df.loc[i, idx] = np.random.randint(1, 6)  # Fill with a random number from 1 to 5

# Save the modified DataFrame to a new CSV file
final_csv_path = '/Users/apple/Desktop/Html/customers_4.csv'
df.to_csv(final_csv_path, index=False)

print(f"Modified data saved successfully at {final_csv_path}.")


Modified data saved successfully at /Users/apple/Desktop/Last_year/IRS/222102535Karim_assignment1/customers_4.csv.


In [ ]:
#practice 
URL='https://www.amazon.com/PlayStation-Gaming-Guide-Black-White/dp/1925638774/ref=sr_1_3?crid=1JXKQMRVCXD0S&dib=eyJ2IjoiMSJ9.dhJaQ6bkhqupJk94JZvRnMgowOgYF_dKZ3jQMrpBS43efzbuNBBmgzD5azdU0H6GgODfO8CWwmDjipOhK5KWBu_HWPKNK5Uz0wrJN3_IWODN8LUiagQb_YeRXgxivl8tpZOifX5xMKkxRiLsLmK4meTwJtDV6VnXZTxhIIGuahEaEs6U7IXPCwq6gBzc698iuwt45jpgh7OPEemD9xN-oWRl83xFqHYylfn_w1xwt1M.0gxJoS8KIfW09__HcH8z96QErkalD9SJDlZqcr2b2zU&dib_tag=se&keywords=playstation+5&qid=1730645063&s=books&sprefix=playstation+5%2Cstripbooks-intl-ship%2C626&sr=1-3#customerReviews'

In [ ]:
reviews1=requests.get(URL,headers=Headers)

In [ ]:
reviews1


In [ ]:
# convert to html
soup = BeautifulSoup(reviews1.content, 'html.parser')

# Find all review divs
reviews = soup.find_all('div', class_='a-section review aok-relative')

# saving data
customer_data = []

# eaxtract each customer and his rating 
for review in reviews:
    # Extract customer name
    name = review.find('span', class_='a-profile-name').get_text(strip=True)
    
    # Extract rating
    rating = review.find('i', {'data-hook': 'review-star-rating'}).find('span').get_text(strip=True)
    
    # store
    customer_data.append({'name': name, 'rating': rating})


for data in customer_data:
    print(f"Customer Name: {data['name']}, Rating: {data['rating']}")


In [ ]:
URL2="https://www.amazon.com/River-City-Girls-Zero-PS5-Limited/dp/B0BR8XF6S7/ref=sr_1_4?crid=1JXKQMRVCXD0S&dib=eyJ2IjoiMSJ9.dhJaQ6bkhqupJk94JZvRnMgowOgYF_dKZ3jQMrpBS43efzbuNBBmgzD5azdU0H6GgODfO8CWwmDjipOhK5KWBu_HWPKNK5Uz0wrJN3_IWODN8LUiagQb_YeRXgxivl8tpZOifX5xMKkxRiLsLmK4meTwJtDV6VnXZTxhIIGuahEaEs6U7IXPCwq6gBzc698iuwt45jpgh7OPEemD9xN-oWRl83xFqHYylfn_w1xwt1M.0gxJoS8KIfW09__HcH8z96QErkalD9SJDlZqcr2b2zU&dib_tag=se&keywords=playstation+5&qid=1730652436&s=books&sprefix=playstation+5%2Cstripbooks-intl-ship%2C626&sr=1-4"

In [ ]:
reviews2=requests.get(URL2,headers=Headers)

In [ ]:
reviews2

In [ ]:
# convert to html
soup = BeautifulSoup(reviews2.content, 'html.parser')

# Find all review divs
reviews = soup.find_all('div', class_='a-section review aok-relative')

# saving data
customer_data = []

# eaxtract each customer and his rating 
for review in reviews:
    # Extract customer name
    name = review.find('span', class_='a-profile-name').get_text(strip=True)
    
    # Extract rating
    rating = review.find('i', {'data-hook': 'review-star-rating'}).find('span').get_text(strip=True)
    
    # store
    customer_data.append({'name': name, 'rating': rating})


for data in customer_data:
    print(f"Customer Name: {data['name']}, Rating: {data['rating']}")


In [ ]:
URL3="https://www.amazon.com/Melania-Trump/dp/1510782699/ref=sr_1_1?crid=3K7E49HWHO3ZK&dib=eyJ2IjoiMSJ9.kTpsfyQRHnXmAZfjAWl-oxt3Gdth497qMdfI_6vDl78K8DS26FcURgzpe-WRDnil5pOX9hVlN-GF1eEkuxm09cuKybIUZgbTBH1TdVrAXU60rHX4gOubkgvuMvdoMRnzNyoyHd6GVXmz7leJIc04ySPq3C8dyOFTTQSv8efB-Q-9x36gMnJYbv_3Gd46m8lKprUD5nWMLmA7I2k3h7JnkdUGppQHkUmiW4vd7XFgZL4.GmNvWyjd-NHg56QTRrYhGqqb_LHxX8cTgIzR_22iNfE&dib_tag=se&keywords=books&qid=1730652918&s=books&sprefix=book%2Cstripbooks-intl-ship%2C356&sr=1-1"

In [ ]:
book1=requests.get(URL3, headers=Headers)

In [ ]:
book1

In [ ]:
# convert to html
soup = BeautifulSoup(book1.content, 'html.parser')

# Find all review divs
reviews = soup.find_all('div', class_='a-section review aok-relative')

# saving data
customer_data = []

# eaxtract each customer and his rating 
for review in reviews:
    # Extract customer name
    name = review.find('span', class_='a-profile-name').get_text(strip=True)
    
    # Extract rating
    rating = review.find('i', {'data-hook': 'review-star-rating'}).find('span').get_text(strip=True)
    
    # store
    customer_data.append({'name': name, 'rating': rating})


for data in customer_data:
    print(f"Customer Name: {data['name']}, Rating: {data['rating']}")
